In [ ]:
import folium
mapa = folium.Map(location=[latitude, longitude], zoom_start=12)

for foto in fotos:
    folium.Marker([foto['latitude'], foto['longitude']], popup=foto['titulo']).add_to(mapa)

mapa.save('mapa_fotos.html')